# <B> # SageMaker Evaluation with Processing </B>
* Container: codna_pytorch_p39

## 학습 작업의 실행 노트북 개요

- SageMaker Training에 SageMaker 실험을 추가하여 여러 실험의 결과를 비교할 수 있습니다.
    - [Evaluation 하기](#Evaluation-하기)

## AutoReload

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 0. Install packages

In [2]:
install_needed = True  # should only be True once
# install_needed = False

In [7]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

Already revised


In [8]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U smdebug sagemaker-experiments
    !{sys.executable} -m pip install -U sagemaker
    
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached sagemaker-2.145.0.tar.gz (714 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.145.0-py2.py3-none-any.whl size=959452 sha256=81ff74c3fb79656f2e078588cc6ebd24018fc9401896eb5262ff45fbe9deb284
  Stored in directory: /home/ec2-user/.cache/pip/wheels/fa/df/70/4cea9477b77a1608e68c3adbbeb5b74b4c0908fc3f8f5735b8
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.143.0
    Uninstalling sagemaker-2.143.0:
      Successfully uninstalled sagemaker-2.143.0


## 1. parameter store 설정

In [4]:
import boto3
from utils.ssm import parameter_store

In [5]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

In [6]:
strBucketName = pm.get_params(key="-".join([strPrefix, "BUCKET"]))
strExecutionRole = pm.get_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"]))

## 2. Dataset

In [7]:
import os

In [8]:
strS3DataPath = f"s3://{strBucketName}/dataset" 
strLocalDataPath = os.path.join(os.getcwd(), "data")

## 3.Evaluation (Processing-job)

In [9]:
import os
import sagemaker
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.processing import ProcessingInput, ProcessingOutput, FrameworkProcessor

* params for training job

In [11]:
# Set to True to enable SageMaker to run locally
local_mode = False

if local_mode:
    
    from sagemaker.local import LocalSession
    
    strInstanceType = "local"
    sagemaker_session = LocalSession()
    #sagemaker_session.config = {'local': {'local_code': True}}    
    strTestDataPath = os.path.join(strLocalDataPath, "test.csv")

else:
    strInstanceType = "ml.m5.xlarge" #"ml.p3.2xlarge"#"ml.g4dn.8xlarge"#"ml.p3.2xlarge", 'ml.p3.16xlarge' , ml.g4dn.8xlarge
    
    sagemaker_session = sagemaker.Session()
    strTestDataPath = os.path.join(strS3DataPath, "test.csv")

nInstanceCount = 1
strProcPrefixPath = "/opt/ml/processing"

strOutputPath = os.path.join(
    "s3://{}".format(strBucketName),
    strPrefix,
    "evaluation",
    "output"
)

strCodeLocation = os.path.join(
    "s3://{}".format(strBucketName),
    strPrefix,
    "evaluation",
    "backup_codes"
)

strS3ModelPath = pm.get_params(key="-".join([strPrefix, "MODEL-PATH"]))
strExperimentName = pm.get_params(key="-".join([strPrefix, "EXPERI-NAME"]))

## You can't override the metric definitions for Amazon SageMaker algorithms. 
# strNumeticRegEx = "([0-9\\.]+)(e-?[[01][0-9])?"
# listMetricDefinitions = [
#     {"Name": "train_loss", "Regex": f"loss={strNumeticRegEx}"},
#     {"Name": "wer", "Regex": f"wer:{strNumeticRegEx}"}
# ]

# dicGitConfig = {
#     'repo': f'https://{pm.get_params(key="-".join([prefix, "CODE_REPO"]))}',
#     'branch': 'main',
#     'username': pm.get_params(key="-".join([prefix, "CODECOMMIT-USERNAME"]), enc=True),
#     'password': pm.get_params(key="-".join([prefix, "CODECOMMIT-PWD"]), enc=True)
# }  

kwargs = {}

In [13]:
print (f'strInstanceType: {strInstanceType}')
print (f'nInstanceCount: {nInstanceCount}')
print (f'sagemaker_session: {sagemaker_session}')
print (f'strExperimentName: {strExperimentName}')
print (f'strTestDataPath: {strTestDataPath}')
print (f'strOutputPath: {strOutputPath}')
print (f'strCodeLocation: {strCodeLocation}')
print (f'strS3ModelPath: {strS3ModelPath}')

strInstanceType: ml.m5.xlarge
nInstanceCount: 1
sagemaker_session: <sagemaker.session.Session object at 0x7f3a1ae4a6a0>
strExperimentName: DJ-SM-IMD-experiments
strTestDataPath: s3://sagemaker-us-east-1-419974056037/dataset/test.csv
strOutputPath: s3://sagemaker-us-east-1-419974056037/DJ-SM-IMD/evaluation/output
strCodeLocation: s3://sagemaker-us-east-1-419974056037/DJ-SM-IMD/evaluation/backup_codes
strS3ModelPath: s3://sagemaker-us-east-1-419974056037/DJ-SM-IMD/training/model-output/DJ-SM-IMD-experiments-0416-06421681627343/output/model.tar.gz


* Define evaluation job (processing job)

In [14]:
evaluation_processor = FrameworkProcessor(
    estimator_cls=XGBoost,
    framework_version="1.3-1",
    image_uri=None,
    role=strExecutionRole,
    instance_type=strInstanceType,
    instance_count=nInstanceCount,
    base_job_name="evaluation", # bucket에 보이는 이름 (pipeline으로 묶으면 pipeline에서 정의한 이름으로 bucket에 보임)
    sagemaker_session=sagemaker_session
)

In [15]:
evaluation_processor.run(
    code="evaluation.py",
    source_dir="source/evaluation",
    inputs=[
        ProcessingInput(
            source=strTestDataPath,
            input_name="test_data",
            destination=os.path.join(strProcPrefixPath, "test")
        ),
        ProcessingInput(
            source=strS3ModelPath,
            input_name="model_weight",
            destination=os.path.join(strProcPrefixPath, "model")
        )
    ],
    outputs=[
        ProcessingOutput(
            source=os.path.join(strProcPrefixPath, "output"),
            output_name='evaluation',
            destination=strOutputPath,
        )
    ],
    #job_name=job_name,
    # experiment_config={
    #     'TrialName': job_name,
    #     'TrialComponentDisplayName': job_name,
    # },
    wait=True
)

INFO:sagemaker:Creating processing-job with name evaluation-2023-04-16-06-46-36-874


........................
[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
#############################################
args.model_path: /opt/ml/processing/model/model.tar.gz
args.test_path: /opt/ml/processing/test/test.csv
args.output_evaluation_dir: /opt/ml/processing/output
****** All folder and files under /opt/ml/processing ****** 
('/opt/ml/processing', ['model', 'output', 'input', 'test'], [])
('/opt/ml/processing/model', [], ['model.tar.gz'])
('/opt/ml/processing/output', [], [])
('/opt/ml/processing/input', ['entrypoint', 'code'], [])
('/opt/ml/processing/input/entrypoint', [], ['runproc.sh'])
('/opt/ml/processing/input/code', ['.ipynb_checkpoints'], ['evaluation.py', 'sourcedir.tar.gz'])
('/opt/ml/processing/input/code/.ipynb_checkpoints', [], ['evaluation-checkpoint.py'])
('/opt/ml/processing/test', [], ['test.csv'])
************************************************* 
/miniconda3/lib/python3.7/site-packages/xgboost/

* save path for evaluation.json

In [16]:
pm.put_params(key="-".join([strPrefix, "EVAL-OUTPUT-PATH"]), value=strOutputPath, overwrite=True)

'Store suceess'